### Modificacion de nuevos videos para la prediccion del modelo

In [3]:
import cv2
import os, os.path
from os import scandir, getcwd
import numpy as np

# Funcion para buscar archivos dentro de una carpeta definida
def ls(ruta = getcwd()):
    return [arch.name for arch in scandir(ruta) if arch.is_file()]

# Setting origin directory and destination directory
origin = 'videos_reales'
destination = 'videos_reales/Modified_Videos'

lista_videos=ls(origin)

# Count number of files in the directory
files_number = len(lista_videos)

# Create destination file if not existing
if not os.path.exists(destination):
    os.makedirs(destination)

# Set maximum number of frames
max_frames = 372

# Work on each video    
for count, video in enumerate(lista_videos):
    video_capture = cv2.VideoCapture(origin + '/' + video)

    frame_width = 32
    frame_height = 18
    frames = int(video_capture.get(7))
    fps = 30

    frame_number = 0
    # Read two frames, last and current, and convert current to gray.
    ret, last_frame = video_capture.read()  # frame 0
    ret, current_frame = video_capture.read() # frame 1
    frame_number += 1
    gray = cv2.cvtColor(current_frame, cv2.COLOR_BGR2GRAY)

    # Set file destination name
    if 'X' in video:
        filename = '/Pred_X%d.mp4' % count
    elif 'Y' in video:
        filename = '/Pred_Y%d.mp4' % count
    else:
        filename = '/Pred_SC%d.mp4' % count
    
    # Set Output
    out = cv2.VideoWriter(destination + filename,
                          cv2.VideoWriter_fourcc('M','P','4','V'),
                          fps,
                          (frame_width, frame_height))
    
    while(True):
        # We want two frames- last and current, so that we can calculate the different between them.
        # Store the current frame as last_frame, and then read a new one
        last_frame = current_frame

        # Read a new frame and convert to grayscale
        ret, current_frame = video_capture.read()
        frame_number += 1

        if ret == False and frame_number < max_frames or frame_number >= max_frames: # Counter to be evaluated here in order to avoid the last 49 frames + go until max_frame with last_frame of the video
            blank_frame = np.zeros((frame_height, frame_width, 3), np.uint8)
            while frame_number <= max_frames+1:
                out.write(blank_frame)
                frame_number += 1
            break
        gray = cv2.cvtColor(current_frame, cv2.COLOR_BGR2GRAY)

        # Find the absolute difference between frames
        diff = cv2.absdiff(last_frame, current_frame)        
        
        # Resize the video
        diff = cv2.resize(diff,(frame_width,frame_height),fx=0,fy=0, interpolation = cv2.INTER_AREA)
        
        # Write the frame into the file defined in 'out'
        out.write(diff)

    # When everything done, release the capture
    video_capture.release()
    out.release()
    
    progreso = "Realizado video nº%s de %s" % (count+1, files_number)
    print (progreso, end="\r")

print("\nVideos nuevos preparados...")

Realizado video nº14 de 14
Videos nuevos preparados...


In [13]:
# Incase of bug, release videos
video_capture.release()
out.release()

In [4]:
# Verificacion que el numero de frame es igual en todos los videos
lista_videos_modificados = ls('videos_reales/Modified_Videos')

dic_frames = {}
for video in lista_videos_modificados:
    video_capture = cv2.VideoCapture('videos_reales/Modified_Videos/' + video)
    frames = int(video_capture.get(7))
    print(video,frames)
    if frames not in dic_frames:
        dic_frames[frames] = 1
    else:
        dic_frames[frames] += 1
    
    video_capture.release()

for clave, valor in dic_frames.items():
    print ("Numero de frames: %s para %s video(s)"   % (clave, valor))

Pred_SC3.mp4 372
Pred_SC7.mp4 372
Pred_SC8.mp4 372
Pred_X0.mp4 372
Pred_X1.mp4 372
Pred_X2.mp4 372
Pred_X4.mp4 372
Pred_X5.mp4 372
Pred_X6.mp4 372
Pred_X9.mp4 372
Pred_Y10.mp4 372
Pred_Y11.mp4 372
Pred_Y12.mp4 372
Pred_Y13.mp4 372
Numero de frames: 372 para 14 video(s)
